# Qwen3 1.7B Training + Inference Notebook (4-bit BitsAndBytes + DoRA) - 50% Test Data

This notebook is identical to TT-8 but uses only 50% of test data for test-time training to reduce runtime while maintaining rule distribution balance.

**Key Difference from TT-8:**
- **Test-Time Training**: Uses only 50% of test data (stratified by 'rule' column)
- **Runtime**: Significantly faster training and inference
- **Distribution**: Maintains same rule distribution as full dataset
- **Quality**: Should maintain similar performance with reduced training time

**Structure (like TT-1 & TT-8):**
- Training: 4-bit BitsAndBytes + DoRA fine-tuning with DeepSpeed
- Inference: vLLM with LoRA adapters for fast prediction generation
- All-in-one: Complete pipeline from training to submission

**Benefits of 4-bit + DoRA + 50% Test Data:**
- **Faster Runtime**: ~50% reduction in test-time training duration
- **Lower VRAM**: ~8-10GB per GPU with 4-bit quantization
- **Better Quality**: DoRA generally outperforms QLoRA
- **Balanced Data**: Stratified sampling preserves rule distribution
- Uses TT-1 structure with TT-6 optimization settings

In [ ]:
# Install dependencies - BitsAndBytes + DoRA setup
!uv pip install --system --no-index --find-links='/kaggle/input/jigsaw-packages2/whls/' 'trl==0.21.0' 'optimum==1.27.0' 'bitsandbytes==0.46.1' 'deepspeed==0.17.4' 'logits-processor-zoo==0.2.1' 'vllm==0.10.0'
!uv pip install --system --no-index --find-links='/kaggle/input/jigsaw-packages2/whls/' 'triton==3.2.0'
!uv pip install --system --no-index --find-links='/kaggle/input/jigsaw-packages2/whls/' 'clean-text'
# Install latest PEFT for DoRA support (ensure v0.10.0+)
!uv pip install --system --no-index -U --no-deps --find-links='/kaggle/input/jigsaw-packages2/whls/' 'peft' 'accelerate' 'datasets'

print("✅ Dependencies installed for 4-bit BitsAndBytes + DoRA setup")
print("📁 Model will be loaded from local Kaggle input")

# 1. Train Qwen3 1.7B with 4-bit Quantization + DoRA (50% Test Data)

In [ ]:
%%writefile constants.py
# Using base Qwen3 1.7B model from Kaggle input (no internet needed)
BASE_MODEL_PATH = "/kaggle/input/qwen-3/transformers/1.7b/1"  # Update this path as needed
LORA_PATH = "qwen3_1.7b_4bit_dora_output_50pct/"  # 4-bit DoRA output path (50% test data)
DATA_PATH = "/kaggle/input/jigsaw-agile-community-rules/"

POSITIVE_ANSWER = "Yes"
NEGATIVE_ANSWER = "No"
COMPLETE_PHRASE = "Answer:"
BASE_PROMPT = '''You are given a comment from reddit and a rule. Your task is to classify whether the comment violates the rule. Only respond Yes/No.'''

print("✅ Using Qwen3 1.7B model from local Kaggle input")
print("🔥 TT-9: Using 50% of test data for faster runtime")

In [ ]:
%%writefile utils.py
import pandas as pd
from datasets import Dataset
from constants import POSITIVE_ANSWER, NEGATIVE_ANSWER, COMPLETE_PHRASE, BASE_PROMPT
import random, numpy as np
random.seed(42)
np.random.seed(42)


def build_prompt(row):
    return f"""
{BASE_PROMPT}

Subreddit: r/{row["subreddit"]}
Rule: {row["rule"]}
Examples:
1) {row["positive_example"]}
{COMPLETE_PHRASE} Yes

2) {row["negative_example"]}
{COMPLETE_PHRASE} No

---
Comment: {row["body"]}
{COMPLETE_PHRASE}"""


def get_dataframe_to_train(data_path):
    """TT-9 style: Uses training data + 50% of test data (stratified by rule) for training"""
    train_dataset = pd.read_csv(f"{data_path}/train.csv")
    test_dataset = pd.read_csv(f"{data_path}/test.csv")
    
    # TT-9 modification: Stratified sampling to maintain rule distribution
    # Sample 50% of test data while preserving rule distribution
    test_dataset_sampled = test_dataset.groupby('rule', group_keys=False).apply(
        lambda x: x.sample(frac=0.5, random_state=42)
    ).reset_index(drop=True)
    
    print(f"📊 Original test dataset: {len(test_dataset)} samples")
    print(f"📊 Sampled test dataset: {len(test_dataset_sampled)} samples (50%)")
    print(f"📊 Rule distribution preserved: {len(test_dataset_sampled['rule'].unique())} unique rules")

    flatten = []

    # ---------- Process training data ----------
    train_df = train_dataset[["body", "rule", "subreddit", "rule_violation",
                              "positive_example_1","positive_example_2",
                              "negative_example_1","negative_example_2"]].copy()

    # Randomly select positive_example and negative_example
    train_df["positive_example"] = np.where(
        np.random.rand(len(train_df)) < 0.5,
        train_df["positive_example_1"],
        train_df["positive_example_2"]
    )
    train_df["negative_example"] = np.where(
        np.random.rand(len(train_df)) < 0.5,
        train_df["negative_example_1"],
        train_df["negative_example_2"]
    )

    # Drop original candidate columns
    train_df.drop(columns=["positive_example_1","positive_example_2",
                           "negative_example_1","negative_example_2"], inplace=True)

    flatten.append(train_df)

    # ---------- Process test data (50% sampled) ----------
    for violation_type in ["positive", "negative"]:
        for i in range(1, 3):
            sub_dataset = test_dataset_sampled[["rule","subreddit",
                                                "positive_example_1","positive_example_2",
                                                "negative_example_1","negative_example_2"]].copy()

            if violation_type == "positive":
                # body uses current positive_example
                body_col = f"positive_example_{i}"
                other_positive_col = f"positive_example_{3-i}"  # other positive
                sub_dataset["body"] = sub_dataset[body_col]
                sub_dataset["positive_example"] = sub_dataset[other_positive_col]
                # negative_example randomly selected
                sub_dataset["negative_example"] = np.where(
                    np.random.rand(len(sub_dataset)) < 0.5,
                    sub_dataset["negative_example_1"],
                    sub_dataset["negative_example_2"]
                )
                sub_dataset["rule_violation"] = 1

            else:  # violation_type == "negative"
                body_col = f"negative_example_{i}"
                other_negative_col = f"negative_example_{3-i}"
                sub_dataset["body"] = sub_dataset[body_col]
                sub_dataset["negative_example"] = sub_dataset[other_negative_col]
                sub_dataset["positive_example"] = np.where(
                    np.random.rand(len(sub_dataset)) < 0.5,
                    sub_dataset["positive_example_1"],
                    sub_dataset["positive_example_2"]
                )
                sub_dataset["rule_violation"] = 0

            # Drop original candidate columns
            sub_dataset.drop(columns=["positive_example_1","positive_example_2",
                                      "negative_example_1","negative_example_2"], inplace=True)

            flatten.append(sub_dataset)

    # Merge all DataFrames
    dataframe = pd.concat(flatten, axis=0)
    dataframe = dataframe.drop_duplicates(ignore_index=True)
    
    print(f"📊 Final training dataset: {len(dataframe)} samples")
    print(f"📊 Training data: {len(train_df)} samples")
    print(f"📊 Test-derived data: {len(dataframe) - len(train_df)} samples")

    return dataframe


def build_dataset(dataframe):
    dataframe["prompt"] = dataframe.apply(build_prompt, axis=1)

    columns = ["prompt"]
    if "rule_violation" in dataframe:
        dataframe["completion"] = dataframe["rule_violation"].map(
            {
                1: POSITIVE_ANSWER,
                0: NEGATIVE_ANSWER,
            }
        )
        columns.append("completion")

    dataframe = dataframe[columns]
    dataset = Dataset.from_pandas(dataframe)
    dataset.to_pandas().to_csv("/kaggle/working/dataset_50pct.csv", index=False)
    return dataset

In [ ]:
%%writefile train.py
import pandas as pd
import torch
from trl import SFTTrainer, SFTConfig
from peft import LoraConfig
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from tqdm.auto import tqdm
from transformers.utils import is_torch_bf16_gpu_available
from utils import build_dataset, get_dataframe_to_train
from constants import DATA_PATH, BASE_MODEL_PATH, LORA_PATH


def main():
    # TT-9 style: Use training data + 50% of test data (stratified)
    dataframe = get_dataframe_to_train(DATA_PATH)
    train_dataset = build_dataset(dataframe)
    
    print(f"Training dataset size: {len(train_dataset)} samples")
    
    # BitsAndBytes 4-bit quantization config for DoRA
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,  # Enable 4-bit quantization
        bnb_4bit_compute_dtype=torch.float16,  # Compute in FP16
        bnb_4bit_use_double_quant=True,  # Use double quantization for better quality
        bnb_4bit_quant_type="nf4"  # Use NF4 quantization (standard for DoRA)
    )
    print("✅ BitsAndBytes 4-bit quantization config created")
    
    # DoRA configuration with TT-6 settings
    lora_config = LoraConfig(
        r=16,  # From TT-6 config
        lora_alpha=32,  # From TT-6 config  
        lora_dropout=0.05,  # From TT-6 config
        bias="none",
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",
        use_dora=True  # Enable DoRA for better quality
    )
    print("✅ DoRA config created with TT-6 settings")
    
    # TT-6 optimized training config for 4-bit + DoRA
    training_args = SFTConfig(
        num_train_epochs=1,  # Keep same as TT-1
        
        # TT-6 batch sizes optimized for 4-bit
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,  # Effective batch size = 4*4*2 = 32
        
        optim="paged_adamw_8bit",  # Keep 8-bit optimizer for memory efficiency
        learning_rate=1e-4,  # Keep high learning rate like TT-1
        weight_decay=0.01,
        max_grad_norm=1.0,
        
        lr_scheduler_type="cosine",
        warmup_ratio=0.03,  # Same as TT-1
        
        bf16=is_torch_bf16_gpu_available(),
        fp16=not is_torch_bf16_gpu_available(),
        dataloader_pin_memory=True,
        
        gradient_checkpointing=True,
        gradient_checkpointing_kwargs={"use_reentrant": False},
    
        save_strategy="no",  # TT-1 style: don't save during training
        report_to="none",
    
        completion_only_loss=True,
        packing=False,
        remove_unused_columns=False,
    )
    print("✅ Training config created with TT-6 optimizations")
    
    # Load model with BitsAndBytes quantization
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_MODEL_PATH,
        quantization_config=quantization_config,
        torch_dtype=torch.float16,
        # Remove device_map="auto" to avoid distributed training conflicts
        trust_remote_code=True,
        local_files_only=True,  # Use only local files (no internet)
    )
    print("✅ Base model loaded with 4-bit quantization")
    
    # Create SFTTrainer
    trainer = SFTTrainer(
        model=base_model,  # Pass loaded model directly
        args=training_args,
        train_dataset=train_dataset,
        peft_config=lora_config,
    )
    
    print("🚀 Starting 4-bit BitsAndBytes + DoRA training (50% test data)...")
    trainer.train()
    
    # Save LoRA adapters
    trainer.save_model(LORA_PATH)
    print(f"✅ 4-bit DoRA adapters saved to: {LORA_PATH}")


if __name__ == "__main__":
    main()

In [ ]:
%%writefile inference.py
import os
os.environ["VLLM_USE_V1"] = "0"

import vllm
import torch
import pandas as pd
from logits_processor_zoo.vllm import MultipleChoiceLogitsProcessor
from vllm.lora.request import LoRARequest
from utils import build_dataset
from constants import BASE_MODEL_PATH, LORA_PATH, DATA_PATH, POSITIVE_ANSWER, NEGATIVE_ANSWER
import random
import multiprocessing as mp


def run_inference_on_device(df_slice):
    """Run vLLM inference on current process visible GPU"""
    llm = vllm.LLM(
        BASE_MODEL_PATH,
        # Remove quantization="gptq" since we're using BitsAndBytes model
        tensor_parallel_size=1,
        gpu_memory_utilization=0.98,
        trust_remote_code=True,
        dtype="half",
        enforce_eager=True,
        max_model_len=2836,
        disable_log_stats=True,
        enable_prefix_caching=True,
        enable_lora=True,
        max_lora_rank=64,
    )

    tokenizer = llm.get_tokenizer()
    mclp = MultipleChoiceLogitsProcessor(tokenizer, choices=[POSITIVE_ANSWER, NEGATIVE_ANSWER])

    test_dataset = build_dataset(df_slice)
    texts = test_dataset["prompt"]

    outputs = llm.generate(
        texts,
        vllm.SamplingParams(
            skip_special_tokens=True,
            max_tokens=1,
            logits_processors=[mclp],
            logprobs=2,
        ),
        use_tqdm=True,
        lora_request=LoRARequest("default", 1, LORA_PATH)
    )

    log_probs = [
        {lp.decoded_token: lp.logprob for lp in out.outputs[0].logprobs[0].values()}
        for out in outputs
    ]
    predictions = pd.DataFrame(log_probs)[[POSITIVE_ANSWER, NEGATIVE_ANSWER]]
    predictions["row_id"] = df_slice["row_id"].values
    return predictions


def worker(device_id, df_slice, return_dict):
    # Limit this process to see only one GPU
    os.environ["CUDA_VISIBLE_DEVICES"] = str(device_id)
    print(f"[Worker {device_id}] Running on GPU {device_id}, data size={len(df_slice)}")

    preds = run_inference_on_device(df_slice)
    return_dict[device_id] = preds


def main():
    test_dataframe = pd.read_csv(f"{DATA_PATH}/test.csv")

    # Randomly select examples (TT-1 style)
    test_dataframe["positive_example"] = test_dataframe.apply(
        lambda row: random.choice([row["positive_example_1"], row["positive_example_2"]]),
        axis=1
    )
    test_dataframe["negative_example"] = test_dataframe.apply(
        lambda row: random.choice([row["negative_example_1"], row["negative_example_2"]]),
        axis=1
    )
    test_dataframe = test_dataframe.drop(
        columns=["positive_example_1", "positive_example_2", "negative_example_1", "negative_example_2"],
        errors="ignore"
    )

    # Split data for parallel processing
    mid = len(test_dataframe) // 2
    df0 = test_dataframe.iloc[:mid].reset_index(drop=True)
    df1 = test_dataframe.iloc[mid:].reset_index(drop=True)

    manager = mp.Manager()
    return_dict = manager.dict()

    # Two parallel processes
    p0 = mp.Process(target=worker, args=(0, df0, return_dict))
    p1 = mp.Process(target=worker, args=(1, df1, return_dict))
    p0.start()
    p1.start()
    p0.join()
    p1.join()

    # Merge results
    predictions = pd.concat([return_dict[0], return_dict[1]], ignore_index=True)

    # Build submission (TT-1 style)
    submission = predictions[["row_id", POSITIVE_ANSWER]].rename(columns={POSITIVE_ANSWER: "rule_violation"})
    rq = submission['rule_violation'].rank(method='average') / (len(submission) + 1)
    submission['rule_violation'] = rq

    submission.to_csv("/kaggle/working/submission.csv", index=False)
    print("✅ Saved submission.csv using Qwen3 1.7B with 4-bit DoRA (50% test data)")


if __name__ == "__main__":
    main()

In [ ]:
%%writefile accelerate_config.yaml
compute_environment: LOCAL_MACHINE
debug: false
deepspeed_config:
  gradient_accumulation_steps: 4  # TT-6 optimized for 4-bit
  gradient_clipping: 1.0
  train_batch_size: 64  # Effective batch size: 4*4*2 = 32, but DeepSpeed sees 64
  train_micro_batch_size_per_gpu: 4  # TT-6 optimized batch size
  
  zero_stage: 2
  offload_optimizer_device: none
  offload_param_device: none
  zero3_init_flag: false
  
  stage3_gather_16bit_weights_on_model_save: false
  stage3_max_live_parameters: 1e8
  stage3_max_reuse_distance: 1e8
  stage3_prefetch_bucket_size: 5e7
  stage3_param_persistence_threshold: 1e5
  
  zero_allow_untested_optimizer: true
  zero_force_ds_cpu_optimizer: false
  
  fp16:
    enabled: true
    loss_scale: 0
    initial_scale_power: 16
    loss_scale_window: 1000
    hysteresis: 2
    min_loss_scale: 1
  
distributed_type: DEEPSPEED
downcast_bf16: 'no'
dynamo_config:
  dynamo_backend: INDUCTOR
  dynamo_use_fullgraph: false
  dynamo_use_dynamic: false
enable_cpu_affinity: false
machine_rank: 0
main_training_function: main
mixed_precision: fp16
num_machines: 1
num_processes: 2
rdzv_backend: static
same_network: true
tpu_env: []
tpu_use_cluster: false
tpu_use_sudo: false
use_cpu: false

In [ ]:
!accelerate launch --config_file accelerate_config.yaml train.py

In [ ]:
!python inference.py

In [ ]:
!head /kaggle/working/submission.csv

In [ ]:
import pandas as pd
submission_df = pd.read_csv('/kaggle/working/submission.csv')
print(f"Submission shape: {submission_df.shape}")
print(f"Submission head:\n{submission_df.head()}")
print(f"Value range: {submission_df['rule_violation'].min():.6f} to {submission_df['rule_violation'].max():.6f}")
print(f"Mean: {submission_df['rule_violation'].mean():.6f}")

# ⚡ Speed Optimization Guide for 2x T4 GPUs (28GB Total VRAM) - TT-9 Edition

## Current Settings Status: ✅ **OPTIMIZED FOR SPEED** with 4-bit + DoRA + 50% Test Data
- **Memory**: 1.7B model (4-bit) + DoRA fits in ~8-10GB per GPU
- **Batch Size**: 4 per device × 4 accumulation = 32 effective batch size
- **Training**: BitsAndBytes 4-bit + DoRA for highest quality
- **Runtime**: ~50% faster than TT-8 due to reduced test data
- **Distribution**: Stratified sampling preserves rule distribution
- **Structure**: TT-1 style (training + inference in one notebook)

## 🚀 TT-9 Runtime Improvements:

### **Training Speed Benefits:**
1. **50% Less Test Data**: Stratified sampling reduces training time significantly
2. **Same Quality**: Rule distribution preserved via stratified sampling
3. **Memory Efficient**: Same VRAM usage as TT-8
4. **Balanced**: All unique rules still represented

### **Additional Speed Optimizations:**
1. **Increase Batch Size**:
   ```python
   per_device_train_batch_size=6,  # Can go higher with reduced data
   gradient_accumulation_steps=3,   # Adjust accordingly
   ```
   
2. **Faster Optimizer**:
   ```python
   optim="adamw_torch_fused",  # Even faster with less data
   ```

3. **Higher Rank** (since training is faster):
   ```python
   r=32,              # Can use higher rank with faster training
   lora_alpha=64,     # Adjust proportionally
   ```

### **Inference Speed (Same as TT-8):**
1. **Higher GPU Utilization**:
   ```python
   gpu_memory_utilization=0.99,  # Increase from 0.98
   ```

2. **Batch Processing**:
   ```python
   # Split into smaller chunks for faster processing
   chunk_size = len(test_dataframe) // 4  # Use 4 chunks instead of 2
   ```

## 💡 **TT-9 Key Advantages:**
1. **Faster Runtime**: ~50% reduction in training time vs TT-8
2. **Same Quality**: Stratified sampling maintains performance
3. **Rule Balance**: All unique rules preserved in training data
4. **Memory Efficient**: Same VRAM usage as TT-8
5. **Time Limit**: Perfect for Kaggle time constraints
6. **TT-1 Structure**: Complete training + inference pipeline

## 📊 **Data Distribution:**
- **Training Data**: Same as TT-8 (all training examples)
- **Test Data**: 50% stratified sample (grouped by 'rule' column)
- **Total Samples**: ~50% of TT-8 training data
- **Rule Coverage**: 100% of unique rules preserved